# Demo: ML-based Collaborative Filtering on Utility Matrix with Reduced Dimensions

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../resype')
%load_ext autoreload
%autoreload 2 

## Prepare data

In [2]:
np.random.seed(202109)
rating_vals = np.hstack([np.arange(1,6), [np.nan]])
userids = np.arange(10)
itemids = np.arange(100)
random_ratings = np.random.choice(rating_vals, size=len(userids)*len(itemids))

In [3]:
transactions = pd.DataFrame(
    {'user_id': userids.repeat(len(itemids)),
     'item_id': itemids.reshape((-1, 1)).repeat(len(userids), axis=1).T.flatten(),
     'rating': random_ratings}).drop_duplicates()

In [4]:
transactions

,user_id,item_id,rating
0,0,0,2.0
1,0,1,NaN
2,0,2,NaN
3,0,3,5.0
4,0,4,4.0
...,...,...,...
995,9,95,NaN
996,9,96,2.0
997,9,97,NaN
998,9,98,5.0


## Load resype

In [5]:
from resype import Resype

In [6]:
re = Resype(transactions)

In [7]:
utility_matrix = re.construct_utility_matrix()
utility_matrix

item_id,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
0,2.0,NaN,NaN,5.0,4.0,4.0,3.0,4.0,4.0,3.0,...,4.0,NaN,1.0,1.0,NaN,4.0,5.0,5.0,1.0,5.0
1,1.0,3.0,5.0,2.0,NaN,5.0,3.0,3.0,1.0,NaN,...,4.0,4.0,3.0,NaN,2.0,1.0,4.0,1.0,3.0,4.0
2,2.0,1.0,2.0,4.0,5.0,2.0,2.0,1.0,1.0,5.0,...,5.0,2.0,5.0,4.0,2.0,4.0,3.0,NaN,2.0,2.0
3,5.0,5.0,4.0,5.0,2.0,4.0,1.0,3.0,1.0,4.0,...,4.0,2.0,3.0,2.0,4.0,3.0,4.0,4.0,1.0,5.0
4,4.0,1.0,2.0,3.0,2.0,2.0,NaN,5.0,NaN,4.0,...,1.0,NaN,NaN,4.0,NaN,3.0,3.0,2.0,NaN,NaN
5,1.0,5.0,NaN,4.0,2.0,4.0,3.0,NaN,2.0,NaN,...,2.0,4.0,1.0,1.0,1.0,5.0,4.0,1.0,3.0,3.0
6,4.0,2.0,2.0,4.0,1.0,5.0,1.0,3.0,3.0,NaN,...,1.0,5.0,5.0,3.0,2.0,2.0,4.0,3.0,2.0,NaN
7,NaN,3.0,2.0,2.0,NaN,5.0,5.0,5.0,4.0,5.0,...,1.0,2.0,4.0,3.0,2.0,3.0,1.0,1.0,2.0,NaN
8,1.0,NaN,2.0,2.0,1.0,2.0,2.0,2.0,NaN,3.0,...,5.0,2.0,3.0,2.0,2.0,4.0,3.0,1.0,1.0,NaN


## Train model using `train_model_svd`

#### Create model object (load from sklearn)

In [8]:
from sklearn.ensemble import RandomForestRegressor
rs_model1 = RandomForestRegressor(random_state=202109)

#### Train model

In [9]:
np.random.seed(202109)

In [10]:
utility_matrix_imputed = re.train_model_svd(
    re.utility_matrix, rs_model1, d=10, return_models=False)

#### Prediction

In [11]:
utility_matrix_imputed

item_id,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
0,2.00,2.99,3.01,5.0,4.00,4.0,3.00,4.00,4.00,3.00,...,4.0,2.50,1.00,1.00,2.81,4.00,5.0,5.0,1.00,5.00
1,1.00,3.00,5.00,2.0,2.31,5.0,3.00,3.00,1.00,3.82,...,4.0,4.00,3.00,2.07,2.00,1.00,4.0,1.0,3.00,4.00
2,2.00,1.00,2.00,4.0,5.00,2.0,2.00,1.00,1.00,5.00,...,5.0,2.00,5.00,4.00,2.00,4.00,3.0,1.9,2.00,2.00
3,5.00,5.00,4.00,5.0,2.00,4.0,1.00,3.00,1.00,4.00,...,4.0,2.00,3.00,2.00,4.00,3.00,4.0,4.0,1.00,5.00
4,4.00,1.00,2.00,3.0,2.00,2.0,2.33,5.00,2.44,4.00,...,1.0,2.76,2.63,4.00,2.18,3.00,3.0,2.0,1.75,3.61
5,1.00,5.00,3.09,4.0,2.00,4.0,3.00,2.86,2.00,3.55,...,2.0,4.00,1.00,1.00,1.00,5.00,4.0,1.0,3.00,3.00
6,4.00,2.00,2.00,4.0,1.00,5.0,1.00,3.00,3.00,3.21,...,1.0,5.00,5.00,3.00,2.00,2.00,4.0,3.0,2.00,3.61
7,2.71,3.00,2.00,2.0,2.91,5.0,5.00,5.00,4.00,5.00,...,1.0,2.00,4.00,3.00,2.00,3.00,1.0,1.0,2.00,3.66
8,1.00,2.91,2.00,2.0,1.00,2.0,2.00,2.00,2.23,3.00,...,5.0,2.00,3.00,2.00,2.00,4.00,3.0,1.0,1.00,3.05


## Train iterative model using `fit`

#### Create model object (load from sklearn)

#### Train model

In [12]:
rs_model2 = RandomForestRegressor(random_state=202109)
re.fit(rs_model2, method='svd', d=10)

#### Prediction

In [13]:
re.utility_matrix_preds

item_id,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
0,2.00,3.05,3.00,5.0,4.00,4.0,3.00,4.00,4.00,3.00,...,4.0,2.54,1.00,1.00,2.80,4.00,5.0,5.00,1.00,5.00
1,1.00,3.00,5.00,2.0,2.41,5.0,3.00,3.00,1.00,3.83,...,4.0,4.00,3.00,2.06,2.00,1.00,4.0,1.00,3.00,4.00
2,2.00,1.00,2.00,4.0,5.00,2.0,2.00,1.00,1.00,5.00,...,5.0,2.00,5.00,4.00,2.00,4.00,3.0,1.93,2.00,2.00
3,5.00,5.00,4.00,5.0,2.00,4.0,1.00,3.00,1.00,4.00,...,4.0,2.00,3.00,2.00,4.00,3.00,4.0,4.00,1.00,5.00
4,4.00,1.00,2.00,3.0,2.00,2.0,2.35,5.00,2.40,4.00,...,1.0,2.73,2.63,4.00,2.18,3.00,3.0,2.00,1.75,3.58
5,1.00,5.00,3.09,4.0,2.00,4.0,3.00,2.84,2.00,3.54,...,2.0,4.00,1.00,1.00,1.00,5.00,4.0,1.00,3.00,3.00
6,4.00,2.00,2.00,4.0,1.00,5.0,1.00,3.00,3.00,3.19,...,1.0,5.00,5.00,3.00,2.00,2.00,4.0,3.00,2.00,3.62
7,2.63,3.00,2.00,2.0,2.94,5.0,5.00,5.00,4.00,5.00,...,1.0,2.00,4.00,3.00,2.00,3.00,1.0,1.00,2.00,3.67
8,1.00,2.88,2.00,2.0,1.00,2.0,2.00,2.00,2.23,3.00,...,5.0,2.00,3.00,2.00,2.00,4.00,3.0,1.00,1.00,3.12
